In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')

types = {'id': int, 'titulo': str, 'descripcion': str, 'tipodepropiedad': str, 'direccion': str,
        'ciudad': str, 'provincia': str, 'antiguedad': np.float32, 'habitaciones': np.float32,
        'garages':np.float32, 'idzona': np.float32, 'lat': np.float32, 'lng': np.float32,
        'gimnasio': bool, 'usosmultiples': bool, 'piscina': bool, 'escuelascercanas':bool,
        'centroscomercialescercanos':bool, 'precio':int}
pd.set_option('display.max_columns', 30)
data = pd.read_csv('../../data/train.csv', dtype = types)

map_df = gpd.read_file('../../data/mexico_map/mexstates.shp')

data.head()

DriverError: ../../data/mexico_map/mexstates.shp: No such file or directory

In [14]:
geo_df = pd.DataFrame()
geo_df['provincia'] = data['provincia']
geo_df['count'] = 1
geo_count = geo_df.groupby('provincia').agg({'count':'sum'})
geo_count.reset_index(inplace = True)
geo_count

,provincia,count
0,Aguascalientes,1753
1,Baja California Norte,3220
2,Baja California Sur,1700
3,Campeche,263
4,Chiapas,1126
5,Chihuahua,4590
6,Coahuila,3695
7,Colima,1008
8,Distrito Federal,58790
9,Durango,2275


In [15]:
provincias_dict = {'Aguascalientes':'Aguascalientes', 'Baja California Norte':'Baja California',
             'Baja California Sur':'Baja California Sur', 'Campeche':'Campeche', 'Chiapas':'Chiapas',
             'Chihuahua':'Chihuahua', 'Coahuila':'Coahuila', 'Colima':'Colima', 'Distrito Federal':'Distrito Federal',
             'Durango':'Durango', 'Edo. de México':'Mexico', 'Guanajuato':'Guanajuato', 'Guerrero': 'Guerrero',
             'Hidalgo':'Hidalgo', 'Jalisco':'Jalisco', 'Michoacán':'Michoacan', 'Morelos':'Morelos',
             'Nayarit':'Nayarit', 'Nuevo León':'Nuevo Leon', 'Oaxaca':'Oaxaca', 'Puebla':'Puebla',
             'Querétaro':'Queretaro', 'Quintana Roo':'Quintana Roo', 'San luis Potosí':'San Luis Potosi',
             'Sinaloa':'Sinaloa', 'Sonora':'Sonora', 'Tabasco':'Tabasco','Tamaulipas':'Tamaulipas',
             'Tlaxcala':'Tlaxcala', 'Veracruz':'Veracruz', 'Yucatán':'Yucatan','Zacatecas':'Zacatecas'}

In [16]:
def gringofy(x):
    return provincias_dict[x]
    
ad = geo_count.rename(columns={'provincia':'ADMIN_NAME'})
ad['ADMIN_NAME'] = ad['ADMIN_NAME'].apply(gringofy)

ad.dtypes

ADMIN_NAME    object
count          int64
dtype: object

In [17]:
merged = map_df.merge(ad, on='ADMIN_NAME')
merged['count'] = np.log(merged['count'])
merged.head()

KeyError: 'ADMIN_NAME'

In [ ]:
vmin, vmax = 120, 220
fig, ax = plt.subplots(1, figsize=(18, 10))

merged.plot(column='count', cmap='Oranges', linewidth=0.8, ax=ax, edgecolor='0.8', legend = True)
ax.set_title('Mapa de provincias segun numero de publicaciones (escala logaritmica)')
ax.axis('off')

In [ ]:
avg_price = pd.DataFrame()
avg_price['provincia'] = data['provincia']
avg_price['precioenmiles'] = data['precio']/1000
avg_price = avg_price.groupby('provincia').agg({'precioenmiles':'mean'})
avg_price.reset_index(inplace = True)
avg_price

In [ ]:
ad = avg_price.rename(columns={'provincia':'ADMIN_NAME'})
ad['ADMIN_NAME'] = ad['ADMIN_NAME'].apply(gringofy)

In [ ]:
merged = map_df.merge(ad, on='ADMIN_NAME')
merged.head()

In [ ]:
vmin, vmax = 120, 220
fig, ax = plt.subplots(1, figsize=(18, 10))

merged.plot(column='precioenmiles', cmap='Greens', linewidth=0.8, ax=ax, edgecolor='0.8', legend = True)
ax.set_title('Mapa de provincias segun precio (k$) de propiedad promedio')
ax.axis('off')

In [ ]:
m_price = pd.DataFrame()
m_price['provincia'] = data['provincia']
m_price['metrostotales'] = data['metrostotales']
m_price['precio'] = data['precio']
m_price['precio_por_m2'] = m_price['precio']/m_price['metrostotales']
m_price
m_price = m_price.groupby('provincia').agg({'precio_por_m2':'mean', 'metrostotales':'mean'})
m_price.reset_index(inplace = True)
m_price

In [ ]:
m_price = m_price.rename(columns={'provincia':'ADMIN_NAME'})
m_price['ADMIN_NAME'] = m_price['ADMIN_NAME'].apply(gringofy)

In [ ]:
merged = map_df.merge(m_price, on='ADMIN_NAME')
merged.head()

In [ ]:
vmin, vmax = 120, 220
fig, ax = plt.subplots(1, figsize=(18, 10))

merged.plot(column='precio_por_m2', cmap='Greens', linewidth=0.8, ax=ax, edgecolor='0.8', legend = True)
ax.set_title('Mapa de provincias segun precio por m2')
ax.axis('off')

In [ ]:
vmin, vmax = 120, 220
fig, ax = plt.subplots(1, figsize=(18, 10))

merged.plot(column='metrostotales', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8', legend = True)
ax.set_title('Mapa de provincias segun superficie total de propiedad promedio')
ax.axis('off')

In [ ]:
antiguedad = pd.DataFrame()
antiguedad['antiguedad'] = data['antiguedad']
antiguedad['provincia'] = data['provincia']
antiguedad = antiguedad.groupby('provincia').agg({'antiguedad':'mean'})
antiguedad.reset_index(inplace = True)
antiguedad

In [ ]:
antiguedad = antiguedad.rename(columns={'provincia':'ADMIN_NAME'})
antiguedad['ADMIN_NAME'] = antiguedad['ADMIN_NAME'].apply(gringofy)

merged = map_df.merge(antiguedad, on='ADMIN_NAME')
merged.head()

In [ ]:
vmin, vmax = 120, 220
fig, ax = plt.subplots(1, figsize=(18, 10))

merged.plot(column='antiguedad', cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8', legend = True)
ax.set_title('Mapa de provincias segun antiguedad de propiedad promedio')
ax.axis('off')

In [ ]:
tipo = pd.DataFrame()
tipo['provincia'] = data['provincia']
tipo['tipodepropiedad'] = data['tipodepropiedad']
tipo['count'] = 1
tipo.head(10)

In [ ]:
def definir_tipo(x):
    tipos_apartamento = ['Apartamento', 'Departamento Compartido']
    if x in ['Casa en condominio', 'Casa', 'Casa uso de suelo', 'Quinta Vacacional']:
        return 'Casa'
    elif x in ['Apartamento', 'Departamento Compartido']:
        return 'Apartamento'
    return 'Otro'

tipo['tipodepropiedad'] = tipo['tipodepropiedad'].apply(definir_tipo)
tipo = tipo.loc[tipo.tipodepropiedad != 'Otro']
tipo.head()

In [ ]:
n_totales = tipo.groupby(['provincia']).agg({'count':'sum'})
n_tipos = tipo.groupby(['provincia', 'tipodepropiedad']).agg({'count':'sum'})
n_tipos.reset_index(inplace = True)
n_totales.reset_index(inplace = True)
n_tipos = n_tipos.merge(n_totales, on='provincia')
n_tipos['count_x'] = n_tipos['count_x']/n_tipos['count_y']
n_tipos.drop(columns=['count_y'], inplace=True)
n_tipos.rename(columns = {'count_x':'porcentaje'}, inplace = True)
n_tipos

In [ ]:
plt.figure(figsize=(12, 6))
plt.suptitle('% de casas y departamentos por provincia', fontsize=18)
sns.factorplot(x='provincia', y='porcentaje', hue='tipodepropiedad', data=n_tipos, kind='bar')
plt.ylabel("%", fontsize = 14)
plt.xlabel("Provincia", fontsize = 14)
plt.xticks(rotation=90)